In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# from AlgoTestFactory import *

# altf1 = AlgoTestFactory([0, 10000], 15, savePath="Combined_4_test2", storeAccuracies=True)

# altf1.run()

In [3]:
from SktWsegRWR_utf8 import *
import pickle
import ProbData
from ProbModels import *
import multiprocessing
import math
import json
import pprint
import csv
from utilities import *
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

Dataloader Started[Prob]...
Dataloader Finished[Prob]...


In [6]:
pb = ProbModels(fullCo_oc_mat = ProbData.fullCo_oc_mat, unigram_counts = ProbData.unigram_counts,
               cng2cngFullMat = ProbData.cng2cngFullMat, cng2index_dict = ProbData.cng2index_dict,
               w2w_samecng_fullmat=ProbData.w2w_samecng_fullmat, samecng_unigram_counts=ProbData.samecng_unigram_counts,
               v2c_fullMat = ProbData.v2c_fullMat)

goodFileDict = pickle.load(open('mergedGood_v4.p', 'rb'))
algo = SktWsegRWR(
            w2w_modelFunc = pb.get_w2w_mat, 
            t2t_modelFunc = pb.get_cng2cng_mat,
            v2c_modelFunc = pb.get_v2c_ranking,
            sameCng_modelFunc = pb.get_w2w_samecng_mat,
            partition=[0.25, 0.25, 0.25, 0.1]
        )

In [5]:
np.set_printoptions(precision=4)
# for f in list(goodFileDict.keys())[1:20]:
# for f in list(goodFileDict.keys())[1:100]:
for f in fs:
#     print('=='*20)
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj, verbose = False)
        if(result != None):
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            ac = Accuracy(result, dcsObj)
            print(f, "\t", ac)

NameError: name 'fs' is not defined

In [22]:
f = '306555.p'
print(goodFileDict[f])
s, d = loadSentence(f, goodFileDict[f])
SeeSentence(s)

../TextSegmentation/Pickle_Files/306555.p
SKT ANALYZE
---------------
svaDarmaH  svargAyAnantyAya ca   
Analyzing: svaDarmaḥ
0 : sva [0, 'sva', 3] 
3 : Darmas [1, 'Darma', 29] 
-------------------------
Analyzing: svargAyAnantyAya
0 : svargAya [2, 'svarga', 109] [3, 'svarga', 111] 
0 : svargAs [4, 'svarga', 80] [5, 'svarga', 40] [6, 'svarga', 39] 
0 : svarga [7, 'svarga', 3] 
0 : svargA [8, 'svarga', 30] 
0 : svar [9, 'svar', 1] 
0 : sva [10, 'sva', 3] 
0 : svA [11, 'sva', 30] 
3 : fk [12, 'fc', 30] 
3 : fk [13, 'fc', 31] [14, 'fc', 29] [15, 'fc', 71] 
4 : gAya [16, 'ga', 109] [17, 'ga', 111] 
4 : gAya [18, 'gA', -32] 
4 : gAs [19, 'ga', 80] [20, 'ga', 40] [21, 'ga', 39] [22, 'go', 80] [23, 'go', 79] 
4 : gAs [24, 'gA', -302] 
4 : ga [25, 'ga', 3] 
4 : gA [26, 'ga', 30] 
5 : ayAn [27, 'aya', 79] 
5 : ayAn [28, 'yA', -49] 
5 : ayA [29, 'aya', 30] 
5 : aya [30, 'aya', 3] 
6 : yAn [31, 'yad', 79] 
6 : yA [32, 'yad', 30] 
7 : AnantyAya [33, 'Anantya', 109] [34, 'Anantya', 111] 
7 : an [35,

In [24]:
fs = [ '306555.p']
def debugSentences(fs):
    np.set_printoptions(precision=4)
    # for f in list(goodFileDict.keys())[1:20]:
    # for f in list(goodFileDict.keys())[1:100]:
    for f in fs:
    #     print('=='*20)
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result, detail = algo.predict(sentenceObj, dcsObj, verbose = True)
            if(result != None):
                solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
                ac = Accuracy(result, dcsObj)
                print(f, "\t", ac)
                display(detail['sentence'])
                display(detail['DCSLemmas'])
                display(result)
#                 print(detail['nodeList'])
                print('Initial Query:', detail['initialQuery'])
                for step in range(detail['steps']):
                    print('-'*30)
                    print('Step:', step)
                    print('-'*30)
                    print('Winner:', detail[str(step)]['winner'])
                    print('Removed:', detail[str(step)]['removed'])
                    print('Query:', detail[str(step)]['updated_query'])
        print('='*30)

debugSentences(fs)

KeyError: '306555.p'

In [25]:
with open('.temp/sandhi_encounters.csv', 'w') as fh:
    fcsv = csv.writer(fh)
    fcsv.writerow(['left', 'right', 'derivations', 'word_left', 'word_right', 'pos_left', 'pos_right'])
np.set_printoptions(precision=6)
# for f in list(goodFileDict.keys())[90:91]:
for f in list(goodFileDict.keys())[20:40]:
# for f in fullfs:
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result = algo.predict(sentenceObj, dcsObj)
        if(result != None):
            ac = Accuracy(result, dcsObj)
            solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
            print(result)
            print('vs')
            print(solution)
            print(f, "\t", ac)

Solution: ['taTA', 'eva', 'ca', 'kf', 'yaTA', 'sAman', 'eva', 'laB']
Solution No Pvb: ['taTA', 'eva', 'ca', 'kf', 'yaTA', 'sAman', 'eva', 'laB']
['ca', 'yaTA', 'sAman', 'eva', 'eva', 'taTA', 'kf', 'laB']
vs
['taTA', 'eva', 'ca', 'kf', 'yaTA', 'sAman', 'eva', 'laB']
413967.p 	 100.0
Solution: ['vega', 'antara', 'tu', 'anyatama', 'kf', 'karman', 'SItala']
Solution No Pvb: ['vega', 'a', 'tu', 'anyatama', 'kf', 'karman', 'SItala']
['tu', 'karman', 'SItala', 'vA', 'kf', 'anta', 'j', 'anyatama', 'rA']
vs
['vega', 'antara', 'tu', 'anyatama', 'kf', 'karman', 'SItala']
190134.p 	 71.42857142857143
Solution: ['kAla', 'gam', 'tad', 'prasad', 'BU', 'ISvara']
Solution No Pvb: ['kAla', 'gam', 'tad', 'sad', 'BU', 'ISvara']
['ISvara', 'tad', 'na', 'ka', 'a', 'sad', 'BU', 'gam', 'ala']
vs
['kAla', 'gam', 'tad', 'prasad', 'BU', 'ISvara']
255542.p 	 83.33333333333333
Solution: ['buDa', 'BArgava', 'SIGra', 'tadvat', 'pAta', 'yadA', 'sTA']
Solution No Pvb: ['buDa', 'BArgava', 'SIGra', 'tadvat', 'pAta', 'ya

In [31]:
# fullfs = ['105587.p','293473.p','379245.p']
fullfs = ['302864.p', '292204.p', '161425.p', '128533.p', '265112.p', '16349.p', '320178.p', '334655.p', '331468.p', '365975.p', '434538.p', '148267.p', '340997.p', '204751.p', '289589.p', '423330.p', '276728.p', '105056.p', '325962.p', '383515.p', '60055.p', '228316.p', '192549.p', '169215.p', '393452.p', '433996.p', '440103.p', '68686.p', '157189.p', '440567.p']

In [200]:
fullfs = ['302864.p', '292204.p', '161425.p', '128533.p', '265112.p', '16349.p', '320178.p', '334655.p', '331468.p', '365975.p', '434538.p', '148267.p', '340997.p', '204751.p', '289589.p', '423330.p', '276728.p', '105056.p', '325962.p', '383515.p', '60055.p', '228316.p', '192549.p', '169215.p', '393452.p', '433996.p', '440103.p', '68686.p', '157189.p', '440567.p']
def GetSomeJSON():
    lowScore = []
    midScore = []
    highScore = []
    np.set_printoptions(precision=4)
#     for i in range(1, 400):
#         f = list(goodFileDict.keys())[i]
#     print(len(fullfs))
    for f in fullfs:
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)

            if(result != None):
                runDetails['file'] = f
                ac = runDetails['accuracy']
#                 print(f, ac)

                # CODE FOR SELECTING FILES 
#                 if(i%100 == 0):
#                     print('Chekpoint:', i)
                if(ac <= 40):
                    lowScore.append(runDetails)
                elif (ac <= 60):
                    if(ac >= 45):
                        midScore.append(runDetails)
                elif (ac <= 100):
                    if(ac >= 80):
                        highScore.append(runDetails)

    print('Lowscore files found:', len(lowScore))
    print('Midscore files found:', len(midScore))
    print('Highscore files found:', len(highScore))

    lowScore = lowScore[0:10]
    midScore = midScore[0:10]
    highScore = highScore[0:10]
    pickle.dump(lowScore, open('lowScore.p', 'wb'))
    pickle.dump(midScore, open('midScore.p', 'wb'))
    pickle.dump(highScore, open('highScore.p', 'wb'))

#=================================================================
GetSomeJSON()

Lowscore files found: 4
Midscore files found: 7
Highscore files found: 9


In [10]:
def MacroAccuracy():
    allAcs = []
    np.set_printoptions(precision=4)
    gfs = list(goodFileDict.keys())
    for i in range(1000):
#     for i in range(10):
        f = gfs[i]
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        if(sentenceObj != None):
            result = algo.predict(sentenceObj, dcsObj, verbose = False)

            if(result != None):
                ac = Accuracy(result, dcsObj)
                allAcs.append(ac)

                # CODE FOR SELECTING FILES 
                if(i%100 == 0):
                    print('Chekpoint:', i)
    allAcs = np.array(allAcs)
    return allAcs


#=================================================================
allAcs = MacroAccuracy()

Chekpoint: 0
Chekpoint: 200
Chekpoint: 300
Chekpoint: 400
Chekpoint: 600
Chekpoint: 700
Chekpoint: 800
Chekpoint: 900


In [11]:

# the histogram of the data
n, bins, patches = plt.hist(allAcs, 40, normed=False)

plt.xlabel('MicroAccuracy')
plt.ylabel('Count')
plt.title('Histogram of MicroAccuracy')
plt.grid(True)
plt.show()

print('Macro Accuracy %:', 100*np.sum(n[(bins[::] > 95)[:-1:]])/np.sum(n))

Macro Accuracy %: 5.78703703704


In [ ]:
def SeeSomeJSON(region = 'mid', files = ['lowScore.p', 'midScore.p', 'highScore.p']):
    np.set_printoptions(precision=3)
    if region == 'low':
        p = pickleFixLoad(files[0])
    elif region == 'mid':
        p = pickleFixLoad(files[1])
    else:
        p = pickleFixLoad(files[2])
#     for pf in p:
#         print(pf['file'])
    n_steps = p[0]['steps']
    for s in range(n_steps):
        
        print('Step:', s)
        print('='*20)
        
        for key, val in p[0][str(s)].items():
            print(key, ":")
            print(val)
            print('-'*20)
        
SeeSomeJSON('low')
print('='*20)
SeeSomeJSON('mid')
print('='*20)
SeeSomeJSON('high')

In [68]:
def ArePredictionsOK():
    with open('.temp/weird.csv', 'w') as fh:
        wcsv = csv.writer(fh)
        wcsv.writerow(['filename', 'sentence', 'solution', 'prediction'])
        np.set_printoptions(precision=4)
        badCount = 0
        for i in range(1, 600):
            f = list(goodFileDict.keys())[i]

            sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
            if(sentenceObj != None):
                result = algo.predict(sentenceObj, dcsObj, verbose = False)
                if(result != None):            
                    solution = [rom_slp(c) for arr in dcsObj.lemmas for c in arr]
                    ac = Accuracy(result, dcsObj)
                    if ac >= 100 and len(result) != len(solution):
                        badCount += 1
                        if(badCount%10 == 0):
                            print("="*40)
                        sline = [f, sentenceObj.sentence, solution, result]
                        wcsv.writerow(sline)
        print(badCount, 'weird predictions encountered.')
        
ArePredictionsOK()

35 weird predictions encountered.


In [9]:
sentenceObj, dcsObj = loadSentence('1.p', '../TextSegmentation/corrected_10to20/1.p')
SeeSentence(sentenceObj)

SKT ANALYZE
---------------
paYca ratnAni muKyAni coparatnacatuzwayam   
Analyzing: paYca
-------------------------
Analyzing: ratnAni
0 : ratnAni [0, 'ratna', 81] [1, 'ratna', 41] 
0 : ratnAni [2, 'ratna', 61] 
-------------------------
Analyzing: muKyAni
0 : muKyAni [3, 'muKya', 81] [4, 'muKya', 41] 
0 : muKyAni [5, 'muKya', 61] 
-------------------------
Analyzing: coparatnacatuzwayam
0 : ca [6, 'ca', 2] 
9 : catuzwayam [7, 'catuzwaya', 31] [8, 'catuzwaya', 69] [9, 'catuzwaya', 71] 
4 : ratna [10, 'ratna', 3] 
1 : uparatna [11, 'uparatna', 3] 
1 : Upa [12, 'vap', -158] 
-------------------------
Analyzing  paYca
0 :  paYca [] []
Analyzing  ratnAni
0 :  ratnAni ['ratna'] [{'noun': ['acc. pl. n.', 'nom. pl. n.']}]
0 :  ratnAni ['ratna'] [{'noun': ['voc. pl. n.']}]
Analyzing  muKyAni
0 :  muKyAni ['muKya'] [{'noun': ['acc. pl. n.', 'nom. pl. n.']}]
0 :  muKyAni ['muKya'] [{'noun': ['voc. pl. n.']}]
Analyzing  coparatnacatuzwayam
0 :  ca ['ca'] [{'indeclinable': ['conj.']}]
9 :  catuzwa

In [8]:
SeeDCS(dcsObj)

DCS ANALYZE
---------------
upacAreRa SIlena rUpayOvanasaMpadA   
[['upacāra'], ['śīla'], ['rūpa', 'yauvana', 'sampad']]
Lemmas: ['upacAra', 'SIla', 'rUpa', 'yOvana', 'sampad']
[['89'], ['91'], ['3', '3', '90']]



In [7]:
def CSVforSupervised():
    np.set_printoptions(precision=4)
    # Clean the csv files
    metPerfFH = open('.temp/metPerfFH.csv', 'w')
    metPerfFH_bin = open('.temp/metPerfFH_bin.csv', 'w')

    metPerfFH.write('w2w,t2t,w2w_samecng,InDCS\n')
    metPerfFH_bin.write('w2w,t2t,w2w_samecng,InDCS\n')

    metPerfFH.close()
    metPerfFH_bin.close()
    ######################
    gfl = list(goodFileDict.keys())
    for i in range(0, 11000):
        f = gfl[i]
        if(i%250 == 0):
            print('Checkpoint:', i)
        sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
        
        if(sentenceObj != None):
            result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)




In [ ]:
np.set_printoptions(precision=6)
jsonBatch = []
for f_ in lowScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/lowScore.p','wb'))
print('33 % Complete')
#################################################################
jsonBatch = []
for f_ in midScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
pickle.dump(jsonBatch, open('.temp/midScore.p','wb'))
print('67 % Complete')
#################################################################
jsonBatch = []
for f_ in highScore[0:10]:
    f = f_[0]
    sentenceObj, dcsObj = loadSentence(f, goodFileDict[f])
    if(sentenceObj != None):
        result, runDetails = algo.predict(sentenceObj, dcsObj, verbose = True)
        runDetails['file'] = f
        jsonBatch.append(runDetails)
print(pprint.pprint(jsonBatch))
pickle.dump(jsonBatch, open('.temp/highScore.p','wb'))
print('100 % Complete')

In [195]:
a = np.array([1,2,3,4,5])
a[:-1:]

array([1, 2, 3, 4])